In [1]:
from prep_data import PrepData
from anchor_boxes import *
from yolo_model import *

import numpy as np
import tensorflow as tf

from tensorflow.python.ops.gen_math_ops import sigmoid, exp

In [2]:
input_shape = (676, 380, 3)
grid_dim = 13
anchor_sizes = [(0.2, 0.1), (0.3, 0.2), (0.4, 0.3), (0.5, 0.4)]
num_anchors = len(anchor_sizes)

In [3]:
p = PrepData()
train, test = p.load_images()
labels = p.connect_labels(train, grid_dim, anchor_sizes, iou_anchor_label)
train, test = p.preprocess_images(train), p.preprocess_images(test)

In [4]:
# swap axes of x and y of the image
train = np.swapaxes(train, 1, 2)
test = np.swapaxes(test, 1, 2)
train.shape

(1001, 676, 380, 3)

In [17]:
def prediction_to_box(prediction, grid_dim, anchor_sizes, i, j, k):
    x, y, w, h = prediction[0], prediction[1], prediction[2], prediction[3]
    
    # Sigmoid and exp transformations to get the box coordinates from normalized cell coordinates
    x = (sigmoid(x) + i) / grid_dim
    y = (sigmoid(y) + j) / grid_dim
    w = anchor_sizes[k][0] * exp(w)
    h = anchor_sizes[k][1] * exp(h)
                
    return x, y, w, h


def custom_loss(grid_dim, anchor_sizes):
    def loss(labels, predictions):
        def process_per_example(args):
            y_true, y_pred = args
            errors = []

            for i in range(grid_dim):
                for j in range(grid_dim):
                    for k in range(len(anchor_sizes)):
                        min_box_index = k * 5
                        max_box_index = min_box_index + 5
                        
                        pred_box_norm = y_pred[i, j, min_box_index:max_box_index]
                        true_box_norm = y_true[i, j, min_box_index:max_box_index]
                        
                        pred_box = prediction_to_box(pred_box_norm[:-1], grid_dim, anchor_sizes, i, j, k)
                        true_box = prediction_to_box(true_box_norm[:-1], grid_dim, anchor_sizes, i, j, k)
                        
                        iou_score = iou_pred_label(true_box, pred_box)
                        iou_loss = tf.abs(iou_score - pred_box_norm[-1])
                        errors.append(iou_loss)
            
            return tf.reduce_mean(tf.square(errors))

        # This maps the function over the first dimension of labels and predictions, effectively iterating over the batch
        batch_loss = tf.map_fn(process_per_example, (labels, predictions), dtype=tf.float32)
        
        # Return the mean loss over the batch
        return tf.reduce_mean(batch_loss)
    
    
    print(f"Loss: {loss}")
    return loss


def iou_metric(y_true, y_pred):
    iou_scores = []
    
    y_true = tf.squeeze(y_true, axis=0)
    y_pred = tf.squeeze(y_pred, axis=0)
    
    for i in range(grid_dim):
        for j in range(grid_dim):
            for k in range(len(anchor_sizes)):
                pred_box_norm = y_pred[i][j][k*5: k*5+5]
                true_box_norm = y_true[i][j][k*5: k*5+5]   

                pred_box = prediction_to_box(pred_box_norm[:-1], grid_dim, anchor_sizes, i, j, k)
                true_box = prediction_to_box(true_box_norm[:-1], grid_dim, anchor_sizes, i, j, k)

                iou_scores.append(iou_pred_label(true_box, pred_box))
                
    return tf.reduce_mean(iou_scores)

In [18]:
model = yolo_model(input_shape, grid_dim, num_anchors)
model.compile(optimizer='adam', loss=custom_loss(grid_dim, anchor_sizes), metrics=['accuracy'])

In [19]:
model.fit(train, labels, epochs=3, batch_size=16)
model.evaluate(test, labels)

Epoch 1/3
Instructions for updating:
Use fn_output_signature instead
